In this chapter, we will cover the following topics:
<ul>
    <li>Generating a leaflet web map with Folium</li>
    <li>Setting up TileStache to serve tiles</li>
    <li>Visualizing DEM data with Three.js</li>
    <li>Draping an orthophoto over a DEM</li>
</ul>

## Introduction

The great part about geospatial analysis is visualization. This chapter is all about showing some ways to visualize your analysis results. Up to this point, we have used QGIS, leaflet, and Openlayers 3 to see our results. Here, we will concentrate on web mapping with some of the newest libraries to publish our data.

Most of this code will mix Python with JavaScript, HTML, and CSS.

<pre><strong>Tip</strong>

An awesome list of visualization techniques and libraries can be found at http://selection.datavisualization.ch/.</pre>


## 10.1. Generating a leaflet web map with Folium

Creating a web map with your own data is becoming easier with every new web mapping library. Folium (http://folium.readthedocs.org/) is a small new Python project that can create a simple web map directly from your Python code, leveraging the leaflet JavaScript mapping library. This is still more than one line, but with under 20 lines of Python code, you can have Folium generate a nice web map for you.

### Getting ready

Folium requires the Jinja2 template engine alongside Pandas for data binding. The nice part about this is that both are simple to install using pip:

<code>
pip install jinja2
pip install pandas
</code>

Instructions on using Pandas are also found in Chapter 1, Setting Up Your Geospatial Python Environment.

### How to do it...

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import folium
import pandas as pd

# define the polygons
states_geojson = r'us-states.json'

# statistic data to connect to our polygons
state_unemployment = r'../www/html/US_Unemployment_Oct2012.csv'

# read the csv statistic data
state_data = pd.read_csv(state_unemployment)

# Let Folium determine the scale
map = folium.Map(location=[48, -102], zoom_start=3, tiles="Stamen Toner")

# create the leaflet map settings
map.geo_json(geo_path=states_geojson, data=state_data,
             columns=['State', 'Unemployment'],
             threshold_scale=[5, 6, 7, 8, 9, 10],
             key_on='feature.id',
             fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Unemployment Rate (%)')

# output the final map file
map.create_map(path='../www/html/ch10-01_folium_map.html')

### How it works...

Folium uses the Jinja2 Python template engine to render the final results and Pandas to bind the CSV statistic data. The code begins with importing and then defining the data sources. The GeoJSON file of the U.S. State polygons will be displayed as a chloropleth map. A choropleth map is one that displays data values that are classified into a defined set of data ranges, usually based on some statistical method. Within the GeoJSON data is a key-filed named id with a value the U.S. State abbreviation code. This id binds the spatial data to the statistic CSV column that also includes a corresponding id field, hence allowing us to connect our two datasets.

Folium then needs to create a map object, setting the map center coordinates alongside a zoom level and a base tile map for our background. In our case, the Stamen Toner tile set is defined.

Next up, we define the vector GeoJSON that is going to appear on top of our background map. We need to pass in the path of our source GeoJSON and the Pandas data frame object that references our CSV file columns, State and Unemployment. Next, we set the linking key value that connects our CSV with the GeoJSON data. The key_on parameter reads the id GeoJSON properties key in the feature array.

Lastly, we set the color brewer to a color we want along with the style. The legend is a D3 legend that's automatically created for us and is scaled via quantiles.

<img src="./50790OS_10_01.jpg" height=400 width=400>

## 10.2. Setting up TileStache to serve tiles

Once you have data and want to get it onto the Web, a server of some sort is required. TileStache, originally developed by Michal Migurski, is a Python tile map server that can pump out vector tiles. Vector tiles are the future of web mapping and make web map applications super fast. In the end, you will have a TileStache instance running and serving up a simple web map.

### Getting ready

A few requirements are needed to get TileStache running on your machine, including Werkzeug, PIL, SimpleJson, and Modestmaps, so we must first install these. Let's start with running our pip install commands like this:

<pre><strong>Note</strong>

Getting TileStache to run on a full-blown server, such as Nginx or Apache, with mod-python is beyond the scope of this book but is highly recommended for production deployment (for more information on this refer to http://modpython.org/). </pre>

<code>
pip install Werkzeug
pip install modestmaps
pip install simplejson
</code>

The Python library called Werkzeug (http://werkzeug.pocoo.org/) is the WSGI server for our test application. Mapnik is not required, but go ahead and install it to view the demo application.

### How to do it...

1. Now let's download the most recent code from GitHub as a ZIP from https://github.com/TileStache/TileStache/archive/master.zip.
    
<pre><strong>Tip</strong>

    Use the command-line git if you have it installed as follows:

<code>$ git clone https://github.com/TileStache/TileStache.git </code></pre>

2. Unpack this into your /ch10/TileStache-master folder.
3. Test and check whether your installation went smoothly by going into your /ch10/TileStache-master/ directory and entering the following command line:

<code>> python tilestache-server.py -c ../tilestache.cfg </code>

4. After running the preceding command, you should see this:

<code>* Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)</code>

5. Now open up your web browser and type in http://localhost:8080/; you should see some simple text stating TileStache belows hello.
6. Next, try to enter http://localhost:8080/osm/0/0/0.png; you will get the following output:
    
<img src="./50790OS_10_02.jpg" width=400 height=400>

    This is the map of the world that you should be able to see.
    
7. To get a live scrollable map around Vancouver, British Colombia, visit <code>http://localhost:8080/osm/preview.html#10/49.1725/-123.0719.</code>

## 10.3. Visualizing DEM data with Three.js

You have a great 3D Digital Elevation Model (DEM) that you may want to view on a web page, so your choices are limited only to your imagination and programming skills. In this little example based on the great work of Bjorn Sandvik, we will explore the methods needed to manipulate a DEM to load a Three.js HTML-based web page.

<pre><strong>Tip</strong>

A great plugin that I would highly recommend for QGIS is the qgis2threejs plugin, written by Minoru Akagi. The Python plugin code is available on GitHub at https://github.com/minorua/Qgis2threejs where you can find a nice gdal2threejs.py converter.</pre>

The resulting 3D DEM mesh can be viewed in your browser:

<img src="./50790OS_10_03.jpg" width=400 height=400>

### Getting ready

We need Jinja2 as our template engine (installed in the first section of this chapter) to create our HTML. The remaining requirements include JavaScript and our 3D DEM data. Our DEM data is from Chapter 7, Raster Analysis, and is located in the /ch07/geodata/dem_3857.dem folder, so if you have not already downloaded all the data and code, do so now.

The gdal_translate GDAL executable is used to convert our DEM into an ENVI .bin 16-bit raster. This raster will contain the elevation values that the threejs library can read to create the 3D mesh.

<pre><strong>Tip</strong>

Using an IDE is not always necessary, but in this case, the PyCharm Pro IDE is helpful since we are using HTML, JavaScript, and Python to create our results. There is also a free PyCharm community edition that I would also recommend but it lacks the HTML, JavaScript, and Jinja2 template support.</pre>

Three.js is available if you have downloaded the /ch10/www/js folder on your machine. If not, do so now and download the entire /ch10/www/ folder. Inside it, you will find the folders needed for the output of HTML and the web templates used by Jinja2.


### How to do it...

1. We'll start by running a subprocess call to generate the needed raster with elevation data for Three.js. Then, we'll step into the HTML template code containing a single Jinja2 variable as follows:



In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import subprocess

from jinja2 import Environment, FileSystemLoader


# Create our DEM

# use gdal_translate command to create an image to store elevation values
# -scale from 0 meters to 2625 meters
#     stretch all values to full 16bit  0 to 65535
# -ot is output type = UInt16 unsigned 16bit
# -outsize is 200 x 200 px
# -of is output format ENVI raster image .bin file type
# then our input .tif with elevation
# followed by output file name .bin
subprocess.call("gdal_translate -scale 0 2625 0 65535 "
                "-ot UInt16 -outsize 200 200 -of ENVI "
                "../../ch07/geodata/dem_3857.tif "
                "../geodata/whistler2.bin")


# create our Jinja2 HTML
# create a standard Jinja2 Environment and load all files
# located in the folder templates
env = Environment(loader=FileSystemLoader(["../www/templates"]))

# define which template we want to render
template = env.get_template("base-3d-map.html")

# path and name of input 16bit raster image with our elevation values
dem_3d = "../../geodata/whistler2.bin"

# name and location of the output HTML file we will generate
out_html = "../www/html/ch10-03_dem3d_map.html"

# dem_file is the variable name we use in our Jinja2 HTML template file
result = template.render(title="Threejs DEM Viewer", dem_file=dem_3d)

# write out our template to the HTML file on disk
with open(out_html,mode="w") as f:
    f.write(result)

2. Our Jinja2 HTML template code only contains one simple variable called {{ dem_3d }} so that you can see what's happening clearly:



In [ ]:
#!/usr/bin/env python
<html lang="en">
<head>
    <title>DEM threejs Browser</title>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, user-scalable=no, minimum-scale=1.0, maximum-scale=1.0">
    <style> body { margin: 0; overflow: hidden; }</style>
</head>
<body>
    <div id="dem-map"></div>
    <script src="../js/three.min.js"></script>
    <script src="../js/TrackballControls.js"></script>
    <script src="../js/TerrainLoader.js"></script>
    <script>

        var width  = window.innerWidth,
            height = window.innerHeight;

        var scene = new THREE.Scene();

        var axes = new THREE.AxisHelper(200);
        scene.add(axes);

        var camera = new THREE.PerspectiveCamera(45, width / height, 0.1, 1000);
        camera.position.set(0, -50, 50);

        var renderer = new THREE.WebGLRenderer();
        renderer.setSize(width, height);

        var terrainLoader = new THREE.TerrainLoader();
        terrainLoader.load('{{ dem_3d }}', function(data) {

            var geometry = new THREE.PlaneGeometry(60, 60, 199, 199);

            for (var i = 0, l = geometry.vertices.length; i < l; i++) {
                geometry.vertices[i].z = data[i] / 65535 * 10;
            }

            var material = new THREE.MeshPhongMaterial({
                color: 0xdddddd,
                wireframe: true
            });

            var plane = new THREE.Mesh(geometry, material);
            scene.add(plane);

        });

        var controls = new THREE.TrackballControls(camera);

        document.getElementById('dem-map').appendChild(renderer.domElement);

        render();

        function render() {
            controls.update();
            requestAnimationFrame(render);
            renderer.render(scene, camera);
        }

    </script>
</body>
</html>

### How it works...

Our gdal_translate does the hard work for us by converting the DEM data into a raster format that Three.js can understand. The Jinja2 template HTML code shows us the required moving parts, starting with three JavaScript files. TerrainLoader.js reads this binary .bin format raster into the Three.js terrain.

Inside our HTML file, the JavaScript code shows how we can go about creating the Three.js scene where the most important part is creating THREE.PlaneGeometry. We assign each geometry.vertices the elevation height in this JavaScript for loop, assigning each vertex the flat plane of the elevation value.

We follow this with MeshPhongMaterial so that we can see the mesh on our screen as a wireframe. To view the resulting HTML file generated, you need to run a local web server and for this, Python comes with SimpleHTTPServer out of the box. This can be run from the command line as the following Python command:

<code>> python -m SimpleHTTPServer 8080</code>

Then, go visit your browser and enter http://localhost:8080/; select the html folder, and then click on the ch10-03_dem3d_map.html file.

<pre><strong>Tip</strong>

Using the PyCharm IDE, you can simply open the HTML file inside PyCharm, move your mouse to the upper right-hand corner of the open file, and select a browser, such as Chrome, to open a new HTML page. PyCharm will automatically start a web server for you and display the 3D terrain in your selected browser.</pre>

## 10.4. Draping an orthophoto over a DEM

This time around, we are going to take our previous recipe to the next level by draping satellite imagery over our DEM to create a truly impressive 3D interactive web map.

<img src="./50790OS_10_04.jpg" height=400 width=400>

You can take a look at other orthophotos from geogratis.ca at http://geogratis.gc.ca/api/en/nrcan-rncan/ess-sst/77618678-421b-4a28-a0a5-b074e5f072ff.html.

### Getting ready

To drape an orthophoto directly over our DEM, we need to make sure that the input DEM and the orthophoto have the same extent and pixel size. For this exercise, you need to complete the previous section and have data available in the /ch10/geodata/092j02_1_1.tif folder. This is the orthophoto that we are going to drape over the DEM.

### How to do it...

1. Let's dive into some code that's full of comments for your enlightenment:



In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import subprocess
from PIL import Image
from jinja2 import Environment, FileSystemLoader


# convert from Canada UTM http://epsg.io/3157/map   to 3857
# transform the orthophto from epsg:3157 to epsg:3857
# cut the orthophoto to same extent of DEM
subprocess.call("gdalwarp -s_srs EPSG:3157 -t_srs EPSG:3857 -overwrite "
                "-te -13664479.091 6446253.250 -13636616.770 6489702.670"
                "/geodata/canimage_092j02_tif/092j02_1_1.tif ../geodata/whistler_ortho.tif")

# convert the new orthophoto into a 200 x 200 pixel image
subprocess.call("gdal_translate -outsize 200 200 "
                "../geodata/whistler_ortho.tif "
                "../geodata/whistler_ortho_f.tif")

# prepare to create new jpg output from .tif
processed_ortho = '../geodata/whistler_ortho_f.tif'
drape_texture = '../../geodata/whistler_ortho_f.jpg'

# export the .tif to a jpg to make is smaller for web using pil
Image.open(processed_ortho).save(drape_texture)

# set Jinja2 env and load folder where templates are located
env = Environment(loader=FileSystemLoader(["../www/templates"]))

# assign template to our HTML file with our variable inside
template = env.get_template( "base-3d-map-drape.html")

# define the original DEM file
dem_3d = "../../geodata/whistler2.bin"

# location of new HTML file to be output
out_html = "../www/html/ch10-04_dem3d_map_drape.html"

# create the new output HTML object and set variable names
result = template.render(title="Threejs DEM Drape Viewer", dem_file=dem_3d,
                         texture_map=drape_texture)

# write the new HTML file to disk
with open(out_html,mode="w") as file:
    file.write(result)

2. Our Jinja2 HTML template file looks like this:



In [ ]:
<html lang="en">
<head>
    <title>DEM threejs Browser</title>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, user-scalable=no, minimum-scale=1.0, maximum-scale=1.0">
    <style> body { margin: 0; overflow: hidden; }</style>
</head>
<body>
    <div id="dem-map"></div>
    <script src="../js/three.min.js"></script>
    <script src="../js/TrackballControls.js"></script>
    <script src="../js/TerrainLoader.js"></script>
    <script>

        var width  = window.innerWidth,
            height = window.innerHeight;

        var scene = new THREE.Scene();
        scene.add(new THREE.AmbientLight(0xeeeeee));
       
        var axes = new THREE.AxisHelper(200);
        scene.add(axes);

        var camera = new THREE.PerspectiveCamera(45, width / height, 0.1, 1000);
        camera.position.set(0, -50, 50);

        var renderer = new THREE.WebGLRenderer();
        renderer.setSize(width, height);

        var terrainLoader = new THREE.TerrainLoader();
        terrainLoader.load('{{ dem_file }}', function(data) {

            var geometry = new THREE.PlaneGeometry(60, 60, 199, 199);

            for (var i = 0, l = geometry.vertices.length; i < l; i++) {
                geometry.vertices[i].z = data[i] / 65535 * 10;
            }

            var material = new THREE.MeshPhongMaterial({
              map: THREE.ImageUtils.loadTexture('{{ texture_map }}')
            });

            var plane = new THREE.Mesh(geometry, material);
            scene.add(plane);

        });

        var controls = new THREE.TrackballControls(camera);
        document.getElementById('dem-map').appendChild(renderer.domElement);
        render();
        function render() {
            controls.update();
            requestAnimationFrame(render);
            renderer.render(scene, camera);
        }

    </script>
</body>
</html>

### How it works...

The main methodology for draping an orthophoto is the same as seen in the previous section, with a slight difference in the way we use the Three.js material rendering.

Data preparation plays the biggest and most important role once again to make things jive together. Inside our Python code, Ch10-04_drapeOrtho.py uses the subprocess call to execute the gdalwarp and gdal_translate command-line tools. Gdalwarp is first used by taking the original orthophoto in EPSG:3157 and converting it to the EPSG:3857 Web Mercator format. At the same time, it also cuts the original raster to the same extent as our DEM input. This extent is achieved by reading the gdalinfo whistler.bin raster command-line call.

After this, we need to cut the raster down to size and make a 200 x 200 pixel image to match our DEM size. This is followed by using PIL to transform the output .tif file into a much smaller .jpg file that's better suited for web presentations and speed.

With the major leg work out of the way, we can use Jinja2 to create our output HTML template and pass in two dem_file, variables pointing to the original DEM. The second variable called texture_map points to the newly created whistler .jpg that's used to drape over the DEM.

The final results are written to the /ch10/www/html/ch10-04_dem3d_map_drape.html folder for you to then open and view in the browser. To view this HTML file, you will need to start a local web server from the /ch10/www/ directory:

<code>> python -m simpleHTTPServer 8080</code>

Then, visit the browser at http://localhost.8080/ and you should see a draped image on the DEM.